# Preprocessing and storing DF in pickle
Perhaps we might Dwh Pat Dn Date later on. 

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import os 
from datetime import timedelta
pd.set_option('display.max_columns', None)

data_path = str(Path(os.getcwd())) + "/data/"

def write_to_pickle(dataframe, name):
    dataframe.to_pickle(data_path + name + ".pickle")
def read_from_pickle(name): 
    return pd.read_pickle(data_path + name + ".pickle")

In [5]:
bio = pd.read_excel('Dropbox/Healthcare work Patric/20170713_Marc Gliomes_Bio.xlsx', index_col = None)
clin = pd.read_excel('Dropbox/Healthcare work Patric/20170713_Marc Gliomes_Clin.xlsx', index_col = None)

In [6]:
b = bio.loc[bio['Pat Nip']==4406078178,'Dwh Prv Date Chir']
c = clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']

#c.iloc[:len(c)]=1
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] = c
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'] 
#print(clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date'].iloc[i])
#clin.loc[clin['Pat Nip']==4406078178,'Clin Chir Date']    
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())
#bio.loc[2,'Dwh Prv Date Chir'].to_pydatetime()-timedelta(days=1)
#clin['Clin Chir Date'] = clin['Clin Chir Date'].apply(lambda x: x.to_pydatetime() + timedelta(days=1))
#bio['Dwh Prv Date Chir'] = bio['Dwh Prv Date Chir'].apply(lambda x: x.to_pydatetime())

Fix the surgery and clinical date so that they match

In [7]:
# Make the two surgery dates match 
# You are taking copies of the dataframe with loc and iloc, so you need to pay attention when you do assignment
# Can definitely remove one or both 'for' loops
for val in np.unique(clin['Pat Nip']): 
    bio_data = bio.loc[bio['Pat Nip']==val,'Dwh Prv Date Chir']
    clin_data = clin.loc[clin['Pat Nip']==val,'Clin Chir Date']
    lc = len(clin_data)
    lb = len(bio_data)
    if lc <=lb:
        for i in range(0,lc):
            clin_data.iloc[i] = bio_data.iloc[i]        
    else: 
        for i in range(0,lb):
            clin_data.iloc[i] = bio_data.iloc[i]
    clin.loc[clin['Pat Nip']==val,'Clin Chir Date'] = clin_data        

In [171]:
df = pd.merge(bio, clin, left_on = ['Pat Nip','Dwh Prv Date Chir'], right_on = ['Pat Nip','Clin Chir Date'],how='outer')
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

Before merging sexes, birthdays and deaths

In [9]:
# Can check before merging the whole thing as well. 
#bio['Pat Sexe'].isnull().value_counts()
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())

False    7456
True      369
Name: Pat Sexe_x, dtype: int64
False    7725
True      100
Name: Pat Date Naissance_x, dtype: int64
True     4921
False    2904
Name: Dwh Pat Date Deces_x, dtype: int64


# Copy all _y columns into _x: sex, birthday and death, clinical histology, grade histology

In [172]:
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Sexe_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Sexe_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Sexe_x'] = x_data

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_x']
    y_data = df.loc[df['Pat Nip']==val,'Pat Date Naissance_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Pat Date Naissance_x'] = x_data    

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Date Deces_x'] = x_data   
    
for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie']
    y_data = df.loc[df['Pat Nip']==val,'Clin Diag Histologie']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Prv Diag Histologie'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Grade']
    y_data = df.loc[df['Pat Nip']==val,'Grade histo']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Grade'] = x_data   

for val in np.unique(df['Pat Nip']): 
    x_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x']
    y_data = df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_y']
    lc = len(x_data)
    for i in range(0,lc):
        check = x_data.iloc[i]
        if pd.isnull(check):
             x_data.iloc[i] = y_data.iloc[i]        
    df.loc[df['Pat Nip']==val,'Dwh Pat Dn Date_x'] = x_data   


In [11]:
# A small improvement
print(df['Pat Sexe_x'].isnull().value_counts())
print(df['Pat Date Naissance_x'].isnull().value_counts())
print(df['Dwh Pat Date Deces_x'].isnull().value_counts())
print(df['Dwh Prv Diag Histologie'].isnull().value_counts())
# Testing for empty values with an example to make sure merge is correct 
#clin.loc[clin['Pat Nip']==1111112776, 'Pat Sexe'].values
#clin['Pat Sexe'].value_counts()
#clin.loc[clin['Pat Sexe'].isnull()==True,'Pat Nip']

False    7493
True      332
Name: Pat Sexe_x, dtype: int64
False    7762
True       63
Name: Pat Date Naissance_x, dtype: int64
True     4908
False    2917
Name: Dwh Pat Date Deces_x, dtype: int64
False    7825
Name: Dwh Prv Diag Histologie, dtype: int64


In [173]:
df.columns

Index(['Dwh Prv Nip', 'Dwh Pat Consentement', 'Pat Nip', 'Pat Sexe_x',
       'Pat Date Naissance_x', 'Dwh Pat Dn Date_x', 'Dwh Pat Date Deces_x',
       'Dwh Prv Date Chir', 'Dwh Prv Diag Histologie', 'Grade', 'Dwh Gen Idh1',
       'Dwh Gen Idh2', 'Dwh Gen Htert C228T', 'Dwh Gen Htert C250T',
       'Dwh Gen P53', 'Dwh Gen Mgmt', 'Dwh Gen Egfr', 'Dwh Gen Mdm2',
       'Dwh Gen Cdk4', 'Dwh Gen P16', 'Dwh Ihc Atrx', 'Dwh Chr Ch1P',
       'Dwh Chr Ch19Q', 'Dwh Chr Ch10P', 'Dwh Chr Ch10Q', 'Dwh Chr Ch9P',
       'Dwh Chr Ch9Q', 'Dwh Pat Tumeur Siege', 'Dwh Pat Tumeur Position',
       'Pat Sexe_y', 'Pat Date Naissance_y', 'Dwh Pat Dn Date_y',
       'Dwh Pat Date Deces_y', 'Clin Diag Histologie', 'Grade histo',
       'Clin Chir Date', 'Clin Evolution', 'Clin Type Chirurgie',
       'Clin Rx Date', 'Dwh Chm Date Debut'],
      dtype='object')

In [174]:
df.rename(columns = {'Pat Nip':'ID','Pat Sexe_x':'Sex','Dwh Pat Dn Date_x':'Dernieres_nouvelles','Pat Date Naissance_x':'DoB','Dwh Pat Date Deces_x':'Death_date',
                      'Dwh Prv Date Chir':'Surgery_date', 'Dwh Prv Diag Histologie':'Tumor_type','Grade':'Tumor_grade',
                      'Dwh Gen Idh1':'Gene_Idh1', 'Dwh Gen Idh2':'Gene_Idh2','Dwh Gen Htert C228T':'Gene_Htert_C228T', 
                      'Dwh Gen Htert C250T':'Gene_Htert_C250T', "Dwh Gen P53":'Gene_P53', 'Dwh Gen Mgmt':'Gene_Mgmt', 
                      'Dwh Gen Egfr':'Gene_Egfr', 'Dwh Gen Mdm2':'Gene_Mdm2','Dwh Gen Cdk4':'Gene_Cdk4',
                      'Dwh Gen P16':'Gene_P16', 'Dwh Ihc Atrx':'Gene_Ihc_Atrx', 'Dwh Chr Ch1P':'Gene_Ch1P',
                      'Dwh Chr Ch19Q':'Gene_Ch19Q', 'Dwh Chr Ch10P':'Gene_Ch10P', 'Dwh Chr Ch10Q':'Gene_Ch10Q', 'Dwh Chr Ch9P':'Gene_Ch9P',
                      'Dwh Chr Ch9Q':'Gene_Ch9Q', 'Dwh Pat Tumeur Siege':'Tumor_Location', 'Dwh Pat Tumeur Position':'Tumor_Position',
                      'Clin Evolution':'Clinical_evolution', 'Clin Type Chirurgie':'Surgery_type',
                      'Clin Rx Date':'Rx_date', 'Dwh Chm Date Debut':'Chemo_start_date'}, inplace = True)
df.drop(['Dwh Pat Dn Date_y','Pat Sexe_y','Dwh Pat Date Deces_y','Pat Date Naissance_y'], axis=1,inplace=True)
df.drop('Dwh Pat Consentement', axis=1, inplace=True)
df.drop('Dwh Prv Nip', axis=1, inplace=True)
df.drop(['Clin Diag Histologie', 'Grade histo','Clin Chir Date',],axis=1,inplace=True)

In [175]:
df.rename(columns = {'Sex':'Gender'},inplace = True) 
#write_to_pickle(df,'post_merge_data')
df.columns

Index(['ID', 'Gender', 'DoB', 'Dernieres_nouvelles', 'Death_date',
       'Surgery_date', 'Tumor_type', 'Tumor_grade', 'Gene_Idh1', 'Gene_Idh2',
       'Gene_Htert_C228T', 'Gene_Htert_C250T', 'Gene_P53', 'Gene_Mgmt',
       'Gene_Egfr', 'Gene_Mdm2', 'Gene_Cdk4', 'Gene_P16', 'Gene_Ihc_Atrx',
       'Gene_Ch1P', 'Gene_Ch19Q', 'Gene_Ch10P', 'Gene_Ch10Q', 'Gene_Ch9P',
       'Gene_Ch9Q', 'Tumor_Location', 'Tumor_Position', 'Clinical_evolution',
       'Surgery_type', 'Rx_date', 'Chemo_start_date'],
      dtype='object')

In [17]:
#df = read_from_pickle('post_merge_data')

In [176]:
df.head(2)

,ID,Gender,DoB,Dernieres_nouvelles,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date
0,4,M,1940-05-22,NaN,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT
1,5,M,1947-10-15,NaN,1999-12-15,1999-05-29,glioblastome IV,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sus-T hémisp,G,TT ini,biopsie,1999-07-12,1999-06-29


# End of merging. Next we deal with nan/empty values

In [20]:
#Multiple entries for the same person, can we treat them independently?
#df.loc[df.ID==4204007149]

In [21]:
df_clean = df.copy(deep=True)

In [177]:
df_clean = df.copy(deep=True)

# Retain observations for which either Idh1 or Idh2 are null but not both - why not remove nulls altogether?
df_clean = df_clean.loc[(df_clean.Gene_Idh2.notnull()) | (df_clean.Gene_Idh1.notnull())]
#df_clean = df_clean[pd.notnull(df.Gene_Idh2)]
#df_clean = df_clean[pd.notnull(df.Gene_Idh1)]

df_clean= df_clean[df_clean.DoB.notnull()]
df_clean = df_clean[df_clean.Gender.notnull()]

# Remove dernieres nouvelles as in this notebook we focus only on patients with a confirmed survival time since surgery
df_clean.drop('Dernieres_nouvelles', axis=1, inplace=True)

In [23]:
# Sanity check
(df_clean.Gene_Idh1.isnull() & df_clean.Gene_Idh2.isnull()).value_counts()

False    7057
dtype: int64

In [24]:
# Remove NaN for surgery dates before creating new variable
df_clean = df_clean[df_clean.Surgery_date.notnull()]
df_clean.Surgery_date.isnull().value_counts()

False    6794
Name: Surgery_date, dtype: int64

In [25]:
# Add age at surgery variable
df_clean['Age_surgery'] = df_clean['Surgery_date'] - df_clean['DoB']

# Time manipulations

In [26]:
df_clean.Death_date.isnull().value_counts()

True     3995
False    2799
Name: Death_date, dtype: int64

In [27]:
# Convert Age at surgery to years
df_clean['Age_surgery'] = df_clean['Age_surgery'] / np.timedelta64(1, 'Y')
df_clean['Age_surgery'] = df_clean['Age_surgery'].round()

Two approaches: 
- we remove null values for death dates
- we deal with null values for life_expectancy later on

In [28]:
# Comment the first line to have the second approach
df_clean = df_clean[df_clean.Death_date.notnull()]
df_clean['life_expectancy'] = df_clean.Death_date - df_clean.Surgery_date
df_clean['life_expectancy'] = df_clean['life_expectancy'] / np.timedelta64(1, 'D')

In [131]:
df_clean_test = df_clean.copy(deep=True)

In [132]:
#Marc rule: aggregate glioblastoma avec oligo IV with GBM; astrocytome gémistocytique (=A2) with astrocytoma grade II
df_clean_test['Tumor_type'].replace({"glioblastome IV": "GBM", "oligo.haut grade III":"oligo",
                                "gliome mixte ana III":"gliome", "oligo. bas grade II": "oligo",
                                "glioblastome avec oligo. IV": "GBM", "gliome mixte II": "gliome",
                                "astrocytome anaplasique III": "astrocytome", "astrocytome bas grade II":"astrocytome",
                                "astrocytome gémistocytique (=A2":"astrocytome", "gangliogliome anaplasique III":"gangliogliome anaplasique"}, inplace=True)
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy
0,4,M,1940-05-22,2001-02-22,2000-05-30,GBM,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non précisé,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0
4,11,M,1958-07-29,1991-11-15,1991-08-14,GBM,4.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PARTIEL,PERDU,NaN,non précisé,non précisé,P1,exérèse,NaT,NaT,33.0,93.0
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,sus-T hémisp,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,sus-T hémisp,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,D,TT ini,exérèse,NaT,NaT,58.0,433.0
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,non précisé,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,non précisé,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0


In [133]:
#Marc comment recode Tumor_Location
df_clean_test['Tumor_Location'].replace({"sus-T hémisp": "A", "sus-T hémisp+profond":"A",
                                "non précisé":"N", "sus-T profond": "A",
                                "Frontal": "A", "tronc cérébral": "B",
                                "cervelet": "B", "ventriculaire":"C",
                                "moelle/rachis":"B", "Rolandique":"A"}, inplace=True)
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy
0,4,M,1940-05-22,2001-02-22,2000-05-30,GBM,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0
4,11,M,1958-07-29,1991-11-15,1991-08-14,GBM,4.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PARTIEL,PERDU,NaN,N,non précisé,P1,exérèse,NaT,NaT,33.0,93.0
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0


In [144]:
#implementing rule per Marc comments
#IDH instead of 'Gene_Idh1', 'Gene_Idh2'
#IDH mutant= 'Gene_Idh1' or 'Gene_Idh2' mutant
#IDHwt= 'Gene_Idh1'wt and  'Gene_Idh2'wt OR 'Gene_Idh1'wt and IDH2 unknown
#Expl : IDH2 mutation are much less frequent than IDH1 mutations (5% of all the mutations)

df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'ALTERE') | (df_clean_test['Gene_Idh2'] == 'ALTERE'), "IDH"] = "mutant"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'] == 'NORMAL'), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'].isnull()), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NORMAL') & (df_clean_test['Gene_Idh2'] == "NC"), "IDH"] = "wt"
df_clean_test.loc[(df_clean_test['Gene_Idh1'] == 'NC') & (df_clean_test['Gene_Idh2'] == 'NC'), "IDH"] = "NC"
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH
0,4,M,1940-05-22,2001-02-22,2000-05-30,GBM,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0,NC
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC
4,11,M,1958-07-29,1991-11-15,1991-08-14,GBM,4.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PARTIEL,PERDU,NaN,N,non précisé,P1,exérèse,NaT,NaT,33.0,93.0,mutant
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0,wt
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0,wt
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0,wt


In [145]:
#now delete IDH1 and IDH2 variables
df_clean_test.drop("Gene_Idh1", axis = 1, inplace=True)
df_clean_test.drop("Gene_Idh2", axis = 1, inplace=True)

In [146]:
#implementing rule per Marc comments
#Tert instead of 'Gene_Htert_C228T', 'Gene_Htert_C250T'
#Tert mutant= 'Gene_Htert_C228T'mut OR 'Gene_Htert_C250T'mut
#Tert wt= 'Gene_Htert_C228T'wt  AND 'Gene_Htert_C250T' wt
df_clean_test.loc[(df_clean_test['Gene_Htert_C228T'] == 'ALTERE') | (df_clean_test['Gene_Htert_C250T'] == 'ALTERE'), "TERT"] = "mutant"
df_clean_test.loc[(df_clean_test['Gene_Htert_C228T'] == 'NORMAL') & (df_clean_test['Gene_Htert_C250T'] == 'NORMAL'), "TERT"] = "wt"
df_clean_test

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH,TERT
0,4,M,1940-05-22,2001-02-22,2000-05-30,GBM,4.0,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0,NC,NaN
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome,3.0,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,A,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,mutant,wt
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome,2.0,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NC,NaN
4,11,M,1958-07-29,1991-11-15,1991-08-14,GBM,4.0,NORMAL,NORMAL,NC,NaN,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PARTIEL,PERDU,NaN,N,non précisé,P1,exérèse,NaT,NaT,33.0,93.0,mutant,wt
5,12,F,1930-12-07,1991-03-13,1990-02-13,GBM,4.0,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,A,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,wt,mutant
7,13,M,1928-11-17,1990-12-20,1989-11-17,GBM,4.0,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,A,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,wt,wt
8,14,F,1934-07-17,1993-03-30,1992-01-22,GBM,4.0,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,D,TT ini,exérèse,NaT,NaT,58.0,433.0,wt,mutant
9,15,M,1936-03-22,1992-03-14,1991-10-23,GBM,4.0,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,A,G,TT ini,exérèse,1991-11-19,NaT,56.0,143.0,wt,mutant
10,16,M,1922-04-18,1991-12-22,1990-11-09,GBM,4.0,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,N,G,TT ini,exérèse,1990-12-01,NaT,69.0,408.0,wt,wt
11,17,M,1924-04-12,1990-04-04,1990-03-28,GBM,4.0,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,N,non précisé,TT ini,exérèse,NaT,NaT,66.0,7.0,wt,mutant


In [147]:
#now delete TERT variables C228T and C250T
df_clean_test.drop("Gene_Htert_C228T", axis = 1, inplace=True)
df_clean_test.drop("Gene_Htert_C250T", axis = 1, inplace=True)

In [148]:
#implementing clinical interaction variable between IDH and TERT
#IDH+ TERT+ == 1 (very good prognosis)
#IDH+ TERT- == 2
#IDH- TERT- == 3
#IDH- TERT+ == 4
df_clean_test.loc[(df_clean_test['IDH'] == 'mutant') & (df_clean_test['TERT'] == 'mutant'), "IDH_TERT"] = 1
df_clean_test.loc[(df_clean_test['IDH'] == 'mutant') & (df_clean_test['TERT'] == 'wt'), "IDH_TERT"] = 2
df_clean_test.loc[(df_clean_test['IDH'] == 'wt') & (df_clean_test['TERT'] == 'wt'), "IDH_TERT"] = 3
df_clean_test.loc[(df_clean_test['IDH'] == 'wt') & (df_clean_test['TERT'] == 'mutant'), "IDH_TERT"] = 4

In [149]:
#implementing clinical 1p19q codeletion variable
#1p19q codel instead of 'Gene_Ch1P', 'Gene_Ch19Q'= in fact not relevant when separated (consider complete loss, no partial loss)
#=What is important is : 1p19q codélétion and this is always associated with IDH
#1p19q codélétion = 'Gene_Ch1P loss AND  'Gene_Ch19Q loss AND IDH mutant (see above)

df_clean_test.loc[(df_clean_test['Gene_Ch1P'] == 'PERDU') & (df_clean_test['Gene_Ch19Q'] == 'PERDU'), "1p19q_codel"] = 1
#problem with following command is that if one value is missing, it will also impute 0 instead of keeping it as missing
#df_clean_test.loc[((df_clean_test['Gene_Ch1P'] != 'PERDU') | (df_clean_test['Gene_Ch19Q'] != 'PERDU'), "1p19q_codel"] = 0
df_clean_test.loc[((df_clean_test['Gene_Ch1P'].notnull()) & (df_clean_test['Gene_Ch19Q'].notnull()) & (df_clean_test['Gene_Ch1P'] != 'PERDU') | (df_clean_test['Gene_Ch19Q'] != 'PERDU')), "1p19q_codel"] = 0

In [150]:
df_clean_test['1p19q_codel'].isnull().value_counts()

False    2796
True        3
Name: 1p19q_codel, dtype: int64

In [151]:
#now delete 1P and 19Q since alone they are each insignificant
df_clean_test.drop("Gene_Ch1P", axis = 1, inplace=True)
df_clean_test.drop("Gene_Ch19Q", axis = 1, inplace=True)

In [153]:
#Marc comment: If IDH is missing, the Dg is GBM, and the patient is >55 y, the probability of 
#IDHmutation is around 5%, so you may also consider to fill this gap
df_clean_test.loc[((df_clean_test['IDH'].isnull()) & (df_clean_test['Tumor_type'] == "GBM") & (df_clean_test['Age_surgery'] > 55.0)), "IDH"] = "wt"

In [95]:
#delete Dwh Chr CH9Q per Marc comments
df_clean_test.drop("Gene_Ch9Q", axis = 1, inplace=True)

In [96]:
#recode CH9P per Marc comments
df_clean_test['Gene_Ch9P'].replace({"PARTIEL":"PERTE", "PERDU": "PERTE"}, inplace=True)

In [97]:
#for Ch10Q, consider just instance where = PERDU, per Marc comments
df_clean_test['Gene_Ch10Q'].replace({"PERDU":"PERTE", "PARTIEL": "NON PERTE", "GAIN": "NON PERTE"}, inplace=True)

In [98]:
#delete Ch10P, per Marc comments
df_clean_test.drop("Gene_Ch10P", axis = 1, inplace=True)

In [122]:
#Marc comment: IDHmut-Tertmut are 1p19codel in 90% of the cases
#impute missing 1p19q codel values per above
df_clean_test.loc[(df_clean_test['IDH_TERT'] == 1) & (df_clean_test['1p19q_codel'].isnull()), "1p19q_codel"] = 1

In [123]:
#Marc comment: IDHmut-Tertwt are ATRX mut in 90% of the cases
#impute missing ATRX values per above
df_clean_test.loc[(df_clean_test['IDH_TERT'] == 2) & (df_clean_test['Gene_Ihc_Atrx'].isnull()), "Gene_Ihc_Atrx"] = "extinction"

In [127]:
#Marc comment: EGFR amplification is extremely rare in IDH mutant
#impute missing EGFR values per above
df_clean_test.loc[(df_clean_test['IDH'] == "mutant") & (df_clean_test['Gene_Egfr'].isnull()), "Gene_Egfr"] = "NORMAL"

#Marc comment: EGFR amplification never occurs in 1p19q codel
#impute missing EGFR values per above
df_clean_test.loc[(df_clean_test['1p19q_codel'] == 1) & (df_clean_test['Gene_Egfr'].isnull()), "Gene_Egfr"] = "NORMAL"

In [129]:
#Marc comment: Loss of chr 10q is rarely associated with IDH mutant (<5%) and almost never (1%) with 1p19q codel
df_clean_test.loc[(df_clean_test['IDH'] == "mutant") & (df_clean_test['Gene_Ch10Q'].isnull()), "Gene_Ch10Q"] = "NON PERTE"
df_clean_test.loc[(df_clean_test['1p19q_codel'] == 1) & (df_clean_test['Gene_Ch10Q'].isnull()), "Gene_Ch10Q"] = "NON PERTE"

In [31]:
# We keep only the first record of patients with multiple surgeries
df_clean_test['ID'].duplicated().value_counts()

False    2204
True      595
Name: ID, dtype: int64

In [32]:
# Sort by ID and Surgery date, in order to keep only earliest surgery date
df_clean_test.sort_values(['ID', 'Surgery_date'], inplace=True)

In [33]:
# Indicate whether duplicate and check sum of them
df_clean_test['multiple_surgeries'] = df_clean_test.duplicated(['ID'])
df_clean_test['multiple_surgeries'].sum()

595

In [34]:
# Reduce to observations that don't have multiple surgeries

df_clean_test.drop_duplicates('ID',keep='first',inplace=True)
df_clean_test.head(5)

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH_TERT,multiple_surgeries
0,4,M,1940-05-22,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000-07-07,NaT,60.0,268.0,NaN,False
2,10,M,1949-11-05,1992-05-15,1992-04-15,gliome mixte ana III,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992-05-01,1992-05-01,42.0,30.0,3.0,False
3,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome bas grade II,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non précisé,non précisé,TT ini,exérèse,1986-04-01,NaT,28.0,2069.0,NaN,False
5,12,F,1930-12-07,1991-03-13,1990-02-13,glioblastome IV,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,sus-T hémisp,G,TT ini,aucune,1990-03-14,NaT,59.0,393.0,4.0,False
7,13,M,1928-11-17,1990-12-20,1989-11-17,glioblastome IV,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,sus-T hémisp,D,TT ini,partielle,1989-12-18,NaT,61.0,398.0,3.0,False


In [35]:
df_final = df_clean_test.copy(deep=True)
df_final.multiple_surgeries.isnull().value_counts()

False    2204
Name: multiple_surgeries, dtype: int64

In [36]:
# They seem to have been datetime objects without the below conversions?
# See here why we need to use loc: 
# https://stackoverflow.com/questions/26724378/pandas-settingwithcopywarning?noredirect=1&lq=1

df_final.loc[:,'Rx_date'] = pd.to_datetime(df_final.Rx_date,errors = 'coerce')
df_final.loc[:,'Chemo_start_date'] = pd.to_datetime(df_final.Chemo_start_date,errors = 'coerce')

df_final.loc[:,'Rx_date'] = df_final['Rx_date'].dt.year
df_final.loc[:,'Chemo_start_date']=df_final.Chemo_start_date.dt.year

In [37]:
# Useful
#for col in df_clean.columns:
 #    print('{0}\n  {1}\n'.format(col,df_clean[col].isnull().value_counts()))

# Merge KPS dataset

In [154]:
# Import KPS dataset to merge it
kps = pd.read_excel('Dropbox/Healthcare work Patric/KPS.xlsx')

In [155]:
# Sort by ID and surgery date, as above
kps.sort_values(['NIP', 'CHIR_DATE'],inplace=True)

In [156]:
# Reduce to single observations as with multiple surgeries from previous dataset
kps.drop_duplicates('NIP',keep='first',inplace=True)

In [157]:
kps.shape

(6772, 6)

In [158]:
# Keep only columns that are of interest or needed for the merge
kps_sorted_final = kps[['NIP', 'ID_EC', 'IK_CLINIQUE', 'CHIR_DATE', 'CHIR_IK_PRE', 'RX_IK_PRE']]

In [159]:
# Merge the two datasets using surgery date and chirurgie date, only the later is not a datetime.
# Use ID for merge as well 
kps.loc[:,'CHIR_DATE'] = pd.to_datetime(kps.CHIR_DATE, errors = 'coerce')

In [160]:
df_final_kps = pd.merge(df_final, kps, how = 'left', left_on = ['ID', 'Surgery_date'], right_on = ['NIP', 'CHIR_DATE'])

In [161]:
len(df_final)

2204

In [48]:
len(df_final_kps)

2204

In [162]:
df_final_kps

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH_TERT,multiple_surgeries,NIP,ID_EC,CHIR_DATE,IK_CLINIQUE,CHIR_IK_PRE,RX_IK_PRE
0,4,M,1940-05-22,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000.0,NaN,60.0,268.0,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN
1,10,M,1949-11-05,1992-05-15,1992-04-15,gliome mixte ana III,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992.0,1992.0,42.0,30.0,3.0,False,1.000000e+01,TTini,1992-04-15,NaN,NaN,NaN
2,11,M,1958-07-29,1991-11-15,1986-03-17,astrocytome bas grade II,2.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,non précisé,non précisé,TT ini,exérèse,1986.0,NaN,28.0,2069.0,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN
3,12,F,1930-12-07,1991-03-13,1990-02-13,glioblastome IV,4.0,NORMAL,NaN,NORMAL,ALTERE,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,PERDU,PERDU,PERDU,NaN,sus-T hémisp,G,TT ini,aucune,1990.0,NaN,59.0,393.0,4.0,False,1.200000e+01,TTini,1990-02-13,NaN,NaN,70.0
4,13,M,1928-11-17,1990-12-20,1989-11-17,glioblastome IV,4.0,NORMAL,NaN,NORMAL,NORMAL,NORMAL,NON METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NaN,NaN,sus-T hémisp,D,TT ini,partielle,1989.0,NaN,61.0,398.0,3.0,False,1.300000e+01,TTini,1989-11-17,NaN,60.0,NaN
5,14,F,1934-07-17,1993-03-30,1992-01-22,glioblastome avec oligo. IV,4.0,NORMAL,NORMAL,NORMAL,ALTERE,NORMAL,NaN,AMPLIFIE,NORMAL,NORMAL,NORMAL,NaN,GAIN,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,D,TT ini,exérèse,NaN,NaN,58.0,433.0,4.0,False,1.400000e+01,TTini,1992-01-22,NaN,70.0,NaN
6,15,M,1936-03-22,1992-03-14,1991-10-23,glioblastome IV,4.0,NORMAL,NORMAL,ALTERE,NORMAL,ALTERE,NON METHYLE,AMPLIFIE,NORMAL,NORMAL,DELETE,NaN,PARTIEL,NON PERTE,NON PERTE,PERDU,NON PERTE,NON PERTE,sus-T hémisp,G,TT ini,exérèse,1991.0,NaN,56.0,143.0,4.0,False,1.500000e+01,TTini,1991-10-23,NaN,80.0,80.0
7,16,M,1922-04-18,1991-12-22,1990-11-09,glioblastome IV,4.0,NORMAL,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NORMAL,NORMAL,NORMAL,DELETE,NaN,PARTIEL,PARTIEL,NaN,PERDU,NON PERTE,NaN,non précisé,G,TT ini,exérèse,1990.0,NaN,69.0,408.0,3.0,False,NaN,NaN,NaT,NaN,NaN,NaN
8,17,M,1924-04-12,1990-04-04,1990-03-28,glioblastome IV,4.0,NORMAL,NaN,NORMAL,ALTERE,NC,METHYLE,NORMAL,NORMAL,NORMAL,NORMAL,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,non précisé,non précisé,TT ini,exérèse,NaN,NaN,66.0,7.0,4.0,False,1.700000e+01,TTini,1990-03-28,NaN,NaN,NaN
9,18,F,1931-06-05,1992-12-18,1992-02-06,astrocytome anaplasique III,3.0,NORMAL,NORMAL,ALTERE,NORMAL,NORMAL,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NaN,NON PERTE,NON PERTE,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992.0,NaN,61.0,316.0,4.0,False,NaN,NaN,NaT,NaN,NaN,NaN


In [169]:
#Clinical_evolution has no missing values. ID_EC is a duplicate from KPS dataset - let's delete it
df_final_kps.drop("ID_EC", axis = 1, inplace=True)

In [168]:
#Marc comment: IK= juste take one value : 'CHIR_IK_PRE', if missing take the average of the two others
df_final_kps['IK'] = df_final_kps['CHIR_IK_PRE']
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].notnull()) & (df_final_kps['RX_IK_PRE'].notnull()), 'IK'] = (df_final_kps['IK_CLINIQUE'] + df_final_kps['RX_IK_PRE']/2)
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].isnull()) & (df_final_kps['RX_IK_PRE'].notnull()), 'IK'] = df_final_kps['RX_IK_PRE'] 
df_final_kps.loc[(df_final_kps['IK'].isnull()) & (df_final_kps['IK_CLINIQUE'].notnull()) & (df_final_kps['RX_IK_PRE'].isnull()), 'IK'] = df_final_kps['IK_CLINIQUE'] 

# Looks good so far 

In [49]:
# Code Rx date into either 1 if date is present, or leave it as missing otherwise
df_final_kps['RX'] = np.where(df_final_kps['Rx_date'].notnull(), 1, np.NaN)

In [50]:
df_final_kps['CHEM'] = np.where(df_final_kps['Chemo_start_date'].notnull(), 1, np.NaN)

In [51]:
#drop NIP as it's duplicate of ID after the merge
#drop multiple surgeries as it was only needed to remove duplicates
#drop CHIR Date as it comes from KPS dataset after merge and is duplicate of Surgery_date
#drop Rx_date and Chemo_start_date
#Final dataset before imputation
#df_final_kps.drop('NIP', axis = 1, inplace=True)
#df_final_kps.drop('multiple_surgeries', axis = 1, inplace=True)
#df_final_kps.drop('CHIR_DATE', axis = 1, inplace=True)
#df_final_kps.drop('Chemo_start_date', axis = 1, inplace=True)
#df_final_kps.drop('Rx_date', axis = 1, inplace=True)

# Keep only observations for which either the death date is observed, or which had RX or CHEMO in 2017
# I find it easier to use dataframe_name[pd.notnull(df.x)]
#df_final_kps = df_final_kps.loc[(df_final_kps.Death_date.notnull())]

In [52]:
df_final_kps.head(2)

,ID,Gender,DoB,Death_date,Surgery_date,Tumor_type,Tumor_grade,Gene_Idh1,Gene_Idh2,Gene_Htert_C228T,Gene_Htert_C250T,Gene_P53,Gene_Mgmt,Gene_Egfr,Gene_Mdm2,Gene_Cdk4,Gene_P16,Gene_Ihc_Atrx,Gene_Ch1P,Gene_Ch19Q,Gene_Ch10P,Gene_Ch10Q,Gene_Ch9P,Gene_Ch9Q,Tumor_Location,Tumor_Position,Clinical_evolution,Surgery_type,Rx_date,Chemo_start_date,Age_surgery,life_expectancy,IDH_TERT,multiple_surgeries,NIP,ID_EC,CHIR_DATE,IK_CLINIQUE,CHIR_IK_PRE,RX_IK_PRE,RX,CHEM
0,4,M,1940-05-22,2001-02-22,2000-05-30,glioblastome IV,4.0,NC,NC,NaN,NaN,NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,autre,G,P1,exérèse,2000.0,NaN,60.0,268.0,NaN,False,NaN,NaN,NaT,NaN,NaN,NaN,1.0,NaN
1,10,M,1949-11-05,1992-05-15,1992-04-15,gliome mixte ana III,3.0,ALTERE,NORMAL,NORMAL,NORMAL,NC,NaN,NORMAL,NaN,NaN,NaN,NaN,NON PERTE,NON PERTE,NON PERTE,PERDU,NON PERTE,NaN,sus-T hémisp,G,TT ini,partielle,1992.0,1992.0,42.0,30.0,3.0,False,10.0,TTini,1992-04-15,NaN,NaN,NaN,1.0,1.0


In [53]:
#df_final_kps['life_expectancy'] = pd.to_datetime(df_final_kps['life_expectancy'])
df_final_kps['life_expectancy'] = df_final_kps['life_expectancy'].round()

# Create Imputation dataset

In [55]:
#df_final_kps.drop(['DoB','Death_date','Surgery_date','Chemo_start_date','Rx_date'],axis=1,inplace=True)
#write_to_pickle(df_final_kps,"preimputation")

In [56]:
# Drop ID so that MICE imputation works
df_final_kps.drop(['ID'], axis=1,inplace=True)

In [ ]:
#checking how many observations we have with different numbers of NA data across variables
no_NAN = df_final_kps.dropna(thresh = 35)
len(no_NAN)

In [ ]:
len(df_final_kps.columns)

# Add it to a csv ready for imputation. Then read the imputed one for one last manipulation

In [60]:
#export dataset for imputation
df_final_kps.to_csv('Dropbox/Healthcare work Patric/imputation_dataset_13022018.csv')

In [ ]:
imputed_dataset.drop(['Unnamed: 0', 'X'], axis = 1, inplace=True)

In [ ]:
imputed_dataset

In [ ]:
imputed_dataset.shape

In [ ]:
final_dataset_analysis.shape

In [ ]:
df_final_kps.shape

In [ ]:
imputed_dataset['ID'] = df_final_kps['ID']

In [ ]:
imputed_dataset.shape

In [ ]:
final_dataset_analysis = imputed_dataset

In [ ]:
final_dataset_analysis.to_csv("final_dataset_analysis.csv")